In [4]:
#누락데이터 처리
import pandas as pd
import seaborn as sns

In [6]:
df = sns.load_dataset('titanic')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [17]:
#deck 컬럼의 값들의 갯수를 세어 봄
print(df['deck'].value_counts())
deck_cnt = df['deck'].value_counts(dropna=False) #Nan 데이터 계산하기
print(deck_cnt)
print()

#isnaull() 메서드로 누락된 데이터 찾기, null이면 True 값이 있으면 False 반환
print(df['deck'].head().isnull())

#isnull()과 sum() 함수를 이용하여 누락된 데이터 찾기
print(df['deck'].isnull().sum())
df.isnull().sum(axis=0) # age, deck, embarked, embark_town = Nan포함

C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: deck, dtype: int64
NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

0     True
1    False
2     True
3    False
4     True
Name: deck, dtype: bool
688


survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [31]:
#반복문으로 Nan 데이터 개수 계산하기
nan_data = df.isnull()
for col in nan_data.columns:
    nan_cnt = nan_data[col].value_counts() #각 열의 Nan 개수 파악
    
    try:
        print(col, ':',nan_cnt[True]) #Nan 값이 존재하면 개수를 출력
        
    except:
        print(col, ':', 0) #Nan값이 없으면 0 출력

# Nan 데이터 확인 -> 어떻게 처리 할 건지?
# Nan 데이터가 있는 컬럼을 삭제?, Nan 데이터가 있는 행을 삭제?
# df.dropna() : axis=1 열을 삭제, axis=0 행을 삭제
# dropna() : Nan이 존재하는 컬럼을 삭제, 갯수가 500이상인 컬럼만 삭제
df_thresh = df.dropna(axis=1, thresh=500)
df_thresh.info()

#age에 Nan이 존재하는 행을 삭제
df_age = df.dropna(subset=['age'], how='any', axis='index')
df_age.info()

# Nan이 존재하는 모든 행을 삭제
nan_col = [df.isnull().sum()>0] #True가 존재하는 컬럼명만 추출
nan_col_names = list(df.columns[nan_col])
df_nan = df.dropna(subset=nan_col_names, how='any', axis='index')
df_nan.info()



survived : 0
pclass : 0
sex : 0
age : 177
sibsp : 0
parch : 0
fare : 0
embarked : 2
class : 0
who : 0
adult_male : 0
deck : 688
embark_town : 2
alive : 0
alone : 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), i

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [37]:
# 누락 데이터 치환 : df.fillna(값 또는 method=ffill/bfill, inplace= True)
print(df['age'].isnull().sum()) #Nan 값 존재 함

# Nan이 입력된 age를 평균 나이로 치환
df_age = df.copy()
df_age['age'].fillna(df['age'].mean(axis=0), inplace=True)
df_age.isnull().sum()

177


survived         0
pclass           0
sex              0
age              0
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [43]:
# 누락 데이터 치환
# embark_town 825~831행 출력
# 가장 빈번하게 나오는 값으로 치환
df.embark_town[825:832]
df['embark_town'].value_counts().idxmax() #가장 빈번하게 발생하는 인덱스 명
df_em = df.copy()
df_em['embark_town'].fillna(df_em['embark_town'].value_counts().idxmax(),
                           inplace=True)
df_em.embark_town[825:832]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
830      Cherbourg
831    Southampton
Name: embark_town, dtype: object

In [45]:
# 누락 데이터 치환 : 이전 데이터로 치환
df_me = df.copy()
df_me['embark_town'].fillna(method='ffill', inplace=True)

print(df_me.embark_town[825:832], df.embark_town[825:832])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
830      Cherbourg
831    Southampton
Name: embark_town, dtype: object 825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
830      Cherbourg
831    Southampton
Name: embark_town, dtype: object


In [51]:
# 중복 데이터 처리 : df.duplicated() -> 중복 여부 확인
df1 = pd.DataFrame({'c1':['a', 'a', 'b', 'a','b'],
                   'c2':[1,1,1,2,2],
                   'c3':[1,1,2,2,2]})
print(df1)
df_dup = df1.duplicated()
df_dup = df1['c1'].duplicated()
df_dup

#중복된 데이터를 제거
df2 = df1.drop_duplicates()
df2

#중복된 컬럼을 제거
df3 = df1.drop_duplicates(subset=['c2','c3'])
df3

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2


In [65]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   age     891 non-null    float64 
 1   fare    891 non-null    float64 
 2   class   891 non-null    category
 3   alive   891 non-null    object  
dtypes: category(1), float64(2), object(1)
memory usage: 22.0+ KB


In [69]:
df_titanic = df.loc[:,['age', 'fare', 'class', 'alive']]
for col in df_titanic.columns:
    if df_titanic[col].dtype == 'float64':
        df_titanic[col].fillna(df_titanic[col].mean(), inplace=True)
    else:
        df_titanic[col].fillna(method='ffill', inplace=True)
print(df_titanic.isnull().sum())
print(len(df_titanic))

df_titanic.drop_duplicates(inplace=True)
print(len(df_titanic))

age      0
fare     0
class    0
alive    0
dtype: int64
891
746


In [72]:
#데이터 표준화 dataset/auto-mpg.csv
df = pd.read_csv('dataset/auto-mpg.csv', header=None)

df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


In [74]:
# 단위 환산 : mpg -> gallon => kg으로 변환
mpg_to_kg = 1.60934 / 3.78541
df['kpl'] = (df['mpg'] * mpg_to_kg).round(2)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.65
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.38
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.65
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,6.80
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,7.23


In [76]:
df['horsepower'].unique() 
# 중간 정수형으로 바꿀 수 없는 ? 발견 -> Nan 변경후 처리

array(['130.0', '165.0', '150.0', '140.0', '198.0', '220.0', '215.0',
       '225.0', '190.0', '170.0', '160.0', '95.00', '97.00', '85.00',
       '88.00', '46.00', '87.00', '90.00', '113.0', '200.0', '210.0',
       '193.0', '?', '100.0', '105.0', '175.0', '153.0', '180.0', '110.0',
       '72.00', '86.00', '70.00', '76.00', '65.00', '69.00', '60.00',
       '80.00', '54.00', '208.0', '155.0', '112.0', '92.00', '145.0',
       '137.0', '158.0', '167.0', '94.00', '107.0', '230.0', '49.00',
       '75.00', '91.00', '122.0', '67.00', '83.00', '78.00', '52.00',
       '61.00', '93.00', '148.0', '129.0', '96.00', '71.00', '98.00',
       '115.0', '53.00', '81.00', '79.00', '120.0', '152.0', '102.0',
       '108.0', '68.00', '58.00', '149.0', '89.00', '63.00', '48.00',
       '66.00', '139.0', '103.0', '125.0', '133.0', '138.0', '135.0',
       '142.0', '77.00', '62.00', '132.0', '84.00', '64.00', '74.00',
       '116.0', '82.00'], dtype=object)

In [79]:
import numpy as np
df['horsepower'].replace('?', np.nan, inplace=True)
df['horsepower'].unique() 

#데이터 타입을 float 변경
df['horsepower'] = df['horsepower'].astype('float')
df['horsepower'].unique() 

array([130., 165., 150., 140., 198., 220., 215., 225., 190., 170., 160.,
        95.,  97.,  85.,  88.,  46.,  87.,  90., 113., 200., 210., 193.,
        nan, 100., 105., 175., 153., 180., 110.,  72.,  86.,  70.,  76.,
        65.,  69.,  60.,  80.,  54., 208., 155., 112.,  92., 145., 137.,
       158., 167.,  94., 107., 230.,  49.,  75.,  91., 122.,  67.,  83.,
        78.,  52.,  61.,  93., 148., 129.,  96.,  71.,  98., 115.,  53.,
        81.,  79., 120., 152., 102., 108.,  68.,  58., 149.,  89.,  63.,
        48.,  66., 139., 103., 125., 133., 138., 135., 142.,  77.,  62.,
       132.,  84.,  64.,  74., 116.,  82.])

In [84]:
#origin
df.origin.unique() #숫자를 category변경 : 나라이름으로 변경
df.origin.replace({1:'USA',2:'EU',3:'JPN'},inplace=True)
df.origin.unique() 

df.origin = df.origin.astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           398 non-null    float64 
 1   cylinders     398 non-null    int64   
 2   displacement  398 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        398 non-null    float64 
 5   acceleration  398 non-null    float64 
 6   model year    398 non-null    category
 7   origin        398 non-null    category
 8   name          398 non-null    object  
 9   kpl           398 non-null    float64 
dtypes: category(2), float64(6), int64(1), object(1)
memory usage: 26.6+ KB


In [85]:
#제조 년도 : model year
df['model year'].unique()

# 숫자를 문자로 문자를 category로 변경
df['model year'] = df['model year'].astype('category')
df.info()
df['model year'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           398 non-null    float64 
 1   cylinders     398 non-null    int64   
 2   displacement  398 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        398 non-null    float64 
 5   acceleration  398 non-null    float64 
 6   model year    398 non-null    category
 7   origin        398 non-null    category
 8   name          398 non-null    object  
 9   kpl           398 non-null    float64 
dtypes: category(2), float64(6), int64(1), object(1)
memory usage: 26.6+ KB


[70, 71, 72, 73, 74, ..., 78, 79, 80, 81, 82]
Length: 13
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]

In [86]:
# 데이터 전처리
# 1. 누락 데이터 처리 : 제거, 치환
# 2. 중복 데이터 처리 : 제거(행), 컬럼...
# 3. 자료 표준화
#    1) 단위환산
#    2) 자료형 변경 : object -> float, object -> category

In [ ]:
#2022.01.09 복습